In [4]:
# %pip install -r requirements.txt

# # PyTorch install for CUDA 12.1
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
# train a machine learning model to predict the genre of a song based on its lyrics with pytorch
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from datasets import load_metric
from datasets import Dataset
from datasets import DatasetDict
import datasets
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

### Setting a device depending on whats available
* ´cuda´ for GPU
* ´cpu´ for CPU
* ´mps´ for Apple silicon

In [2]:
# set random seeds to make sure results are reproducible
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# set device to cuda or mps if available

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(device)

cuda


### Split data in train, test and validation

In [3]:
# load the dataset
raw_datasets = load_dataset("csv", data_files="data.csv", split="train")

# change column name lyrics to sentence and playlist_genre to label
raw_datasets = raw_datasets.rename_column("lyrics", "sentence")
raw_datasets = raw_datasets.rename_column("playlist_genre", "label")

# Split the dataset into train, validation, and test sets
train_testvalid = raw_datasets.train_test_split(test_size=0.2, seed=SEED)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=SEED)

# Assign the resulting datasets to variables
train_dataset = train_testvalid["train"]
valid_dataset = test_valid["train"]
test_dataset = test_valid["test"]

# Now you have train_dataset, valid_dataset, and test_dataset
raw_datasets = {
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset,
}
raw_datasets = datasets.DatasetDict(raw_datasets)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence'],
        num_rows: 155966
    })
    validation: Dataset({
        features: ['label', 'sentence'],
        num_rows: 19496
    })
    test: Dataset({
        features: ['label', 'sentence'],
        num_rows: 19496
    })
})

### Tokenize the data

In [4]:
model_name = "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Function to tokenize a single row
def tokenize_function(row):
    try:
        # Ensure that 'lyrics' is always a string. Replace non-strings with a placeholder.
        lyrics = [
            str(lyric) if isinstance(lyric, str) else "" for lyric in row["sentence"]
        ]
        return tokenizer(lyrics, truncation=True, padding="max_length", max_length=512)
    except Exception as e:
        print(f"Error tokenizing row: {row}")
        print(f"Exception: {e}")
        return None


# Apply tokenization to each subset
tokenized_train = raw_datasets["train"].map(tokenize_function, batched=True)
tokenized_validation = raw_datasets["validation"].map(tokenize_function, batched=True)
tokenized_test = raw_datasets["test"].map(tokenize_function, batched=True)

# Combine back into a DatasetDict
tokenized_datasets = DatasetDict(
    {
        "train": tokenized_train,
        "validation": tokenized_validation,
        "test": tokenized_test,
    }
)

# Check the result
tokenized_datasets

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DatasetDict({
    train: Dataset({
        features: ['label', 'sentence', 'input_ids', 'attention_mask'],
        num_rows: 155966
    })
    validation: Dataset({
        features: ['label', 'sentence', 'input_ids', 'attention_mask'],
        num_rows: 19496
    })
    test: Dataset({
        features: ['label', 'sentence', 'input_ids', 'attention_mask'],
        num_rows: 19496
    })
})

In [5]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Implement evaluation metrics

In [ ]:
# Load the metric function
accuracy = load_metric("accuracy")
precision = load_metric("precision")
recall = load_metric("recall")
f1 = load_metric("f1")


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    acc = accuracy_score(labels, preds)

    # Calculate precision, recall, and f1
    prec, rec, f1_score, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1_score,
    }

### Start training the model

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=12,
    weight_decay=0.01,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# train from latest checkpoint
trainer.train(resume_from_checkpoint=True)

 78%|███████▊  | 91500/116976 [01:50<13:13, 32.11it/s]   

{'loss': 0.084, 'learning_rate': 4.355765285186706e-06, 'epoch': 9.39}


 79%|███████▊  | 92000/116976 [03:38<1:29:20,  4.66it/s]

{'loss': 0.0879, 'learning_rate': 4.270277663794283e-06, 'epoch': 9.44}


 79%|███████▉  | 92500/116976 [05:26<1:27:22,  4.67it/s]

...

100%|█████████▉| 116500/116976 [1:34:28<01:39,  4.80it/s]

{'loss': 0.0536, 'learning_rate': 8.13842155655861e-08, 'epoch': 11.95}


                                                         
100%|██████████| 116976/116976 [1:37:41<00:00, 19.96it/s]

{'eval_loss': 1.8341795206069946, 'eval_accuracy': 0.7761592121460813, 'eval_precision': 0.7763562434187938, 'eval_recall': 0.7761592121460813, 'eval_f1': 0.7761422177522612, 'eval_runtime': 92.0434, 'eval_samples_per_second': 211.813, 'eval_steps_per_second': 26.477, 'epoch': 12.0}
{'train_runtime': 5861.1948, 'train_samples_per_second': 319.319, 'train_steps_per_second': 19.958, 'train_loss': 0.01489559734361701, 'epoch': 12.0}


TrainOutput(global_step=116976, training_loss=0.01489559734361701, metrics={'train_runtime': 5861.1948, 'train_samples_per_second': 319.319, 'train_steps_per_second': 19.958, 'train_loss': 0.01489559734361701, 'epoch': 12.0})

In [ ]:
# save model
# trainer.save_model("genre_model")

### Evaluate the model

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("./genre_model")

# trainer from checkpoint
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [9]:
trainer.evaluate()

100%|██████████| 2437/2437 [03:51<00:00, 10.54it/s]


{'eval_loss': 1.8341795206069946,
 'eval_accuracy': 0.7761592121460813,
 'eval_precision': 0.7763562434187938,
 'eval_recall': 0.7761592121460813,
 'eval_f1': 0.7761422177522612,
 'eval_runtime': 231.6861,
 'eval_samples_per_second': 84.148,
 'eval_steps_per_second': 10.519}

In [11]:
trainer.evaluate(eval_dataset=tokenized_datasets['test'])

100%|██████████| 2437/2437 [01:32<00:00, 26.35it/s]


{'eval_loss': 1.8546456098556519,
 'eval_accuracy': 0.7769798933114485,
 'eval_precision': 0.7770985621765307,
 'eval_recall': 0.7769798933114485,
 'eval_f1': 0.7769661126113228,
 'eval_runtime': 92.5364,
 'eval_samples_per_second': 210.685,
 'eval_steps_per_second': 26.336}

In [12]:
trainer.evaluate(eval_dataset=tokenized_datasets['train'])

100%|██████████| 19496/19496 [12:33<00:00, 25.89it/s]


{'eval_loss': 0.03179062902927399,
 'eval_accuracy': 0.9851570214021005,
 'eval_precision': 0.98521158918168,
 'eval_recall': 0.9851570214021005,
 'eval_f1': 0.9851423452199055,
 'eval_runtime': 753.2059,
 'eval_samples_per_second': 207.07,
 'eval_steps_per_second': 25.884}

### Make predictions on some examples

In [10]:
def predict_genre(sentence):
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    outputs = model(**inputs)
    # print(outputs.logits)
    genre = outputs.logits.argmax(-1)
    if genre == 0:
        return "pop"
    elif genre == 1:
        return "rap"
    elif genre == 2:
        return "rock"
    elif genre == 3:
        return "r&b"
    elif genre == 4:
        return "latin"
    elif genre == 5:
        return "edm"
    return 0

# Eminem - Rap God | RAP
print("Eminem - Rap God | RAP\n", predict_genre("Look, I was gonna go easy on you not to hurt your feelings.")) # is in training data

# random spanish sentence
print("Tu tienes un gato muy bonito\n", predict_genre("Tu tienes un gato muy bonito"))

# Paul Damixie x SERE - You Got Me Like | POP
print("Paul Damixie x SERE - You Got Me Like | POP\n", predict_genre("I tell myself that I'll be better off without you, but you and I know that's a lie. And I can't get you out of my mind. It's like you've got me hypnotized."))
# is not in training data

# compare Japanese with Latin letters and Japanese characters
print("watashi wa raiku desu\n", predict_genre("watashi wa raiku desu"))
print("私はライクです\n", predict_genre("私はライクです"))

Eminem - Rap God | RAP
 rap
Tu tienes un gato muy bonito
 latin
Paul Damixie x SERE - You Got Me Like | POP
 edm
watashi wa raiku desu
 rock
私はライクです
 rock
